# Ristretto Index
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mygs/koffee/blob/master/nespresso/notebooks/ristretto_index.ipynb)

In [1]:
import pandas as pd
import os
import json
import re
import time
from bs4 import BeautifulSoup
import requests
import pickle
import folium
import geopandas
from datetime import datetime
from IPython.display import Audio, display, HTML

display(HTML("<style>audio{display:none}</style>"))
ALARM=Audio(url='https://sound.peal.io/ps/audios/000/021/856/original/youtube_21856.mp3', autoplay=True)


API_KEY=open("getgeoapi.cfg", "r").read()
GETGEOAPI_URL="https://api.getgeoapi.com/v2/currency/convert?api_key="+API_KEY+"&from={}&to=USD&amount=1&format=json"
nespresso = pd.read_csv("../nespresso_ristretto.csv")
with open('ristretto_20220207.pkl', 'rb') as handle:
    ristretto_20220207 = pickle.load(handle)
nespresso.head()

,region,country,flag_code,alpha_3,fx,ristretto_url
0,Africa,Mauritius,im,MUS,MUR,https://www.buynespresso.com/mu_en/ispirazione...
1,Africa,Morocco,ma,MAR,MAD,https://ma.buynespresso.com/ma_en/ispirazione-...
2,Africa,South Africa,za,ZAF,ZAR,https://za.buynespresso.com/za_en/coffee/origi...
3,Africa,Egypt,eg,EGY,EGP,https://www.buynespresso.com/eg_en/ispirazione...
4,Africa,Ivory Coast,ci,CIV,XOF,https://www.buynespresso.com/ci_en/ispirazione...


In [2]:
def get_nespresso_price(url):
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    try:
        script = soup.find("script",text=re.compile(r".*internationalId.*"))
        if script:
            script_string = script.text.replace('\\n', ' ')
            json_object = json.loads(re.findall('window.ui.push\((.*?)\);\n', script_string)[0])
            return float(json_object["configuration"]["eCommerceData"]["product"]["price"])
        else:
            script = soup.find("script",text=re.compile(r".*Magento_Catalog\/js\/product\/view\/provider.*"))
            if script:
                json_object = json.loads(script.text.replace('\\n', ' '))
                data_items = json_object["*"]["Magento_Catalog/js/product/view/provider"]["data"]["items"]

                for k,item in data_items.items():
                    return float(item["price_info"]["regular_price"])
                return "not found"
            else:
                script = soup.find("script",text=re.compile(r".*aepc_pixel_events.*"))
                if script:
                    json_object = json.loads(re.findall('var aepc_pixel_events = (.*?);\n', script.text)[0])
                    return float(json_object["standard_events"]["ViewContent"][0]["params"]["value"])
                else:
                    script = soup.find("script",text=re.compile(r".*priceSpecification.*"))
                    if script:
                        json_object = json.loads(script.text.replace('\\n', ' '))
                        return float(json_object['@graph'][1]["offers"][0]["price"])
                    else:
                        script = soup.find("script",text=re.compile(r".*Viewed Product.*"))
                        if script:
                            json_object = json.loads(re.findall('"Viewed Product",(.*?)\);', script.text)[0])
                            return float(json_object["price"])
                        else:
                            script = soup.find("script",text=re.compile(r".*productID.*"))
                            if script:
                                json_object = json.loads(script.text.replace('\n', ' '))
                                return float(json_object["offers"]["price"])
                            else:
                                script = soup.find("script",text=re.compile(r".*productBasePriceTaxIncl.*"))
                                if script:
                                    price = re.findall('var productBasePriceTaxIncl = (.*?);\n', script.text)[0]
                                    return float(price)
                                else:
                                    product_entity = soup.find("input",{"id": "productEntity"})
                                    if product_entity:
                                        json_object = json.loads(product_entity["value"])
                                        return float(json_object["priceFinal"])
                                    else:
                                        price_tag = soup.find("span",{"class": "nes-capsule-header-price"})
                                        if price_tag:
                                            return float(price_tag.text.replace('€', ''))
                                        else:
                                            price_tag = soup.find("span",{"class": "product__info__price"})
                                            if price_tag:
                                                return float(price_tag.text.replace('₱', ''))
                                            else:
                                                price_tag = soup.find("p",{"class": "product-price"})
                                                if price_tag:
                                                    return float(price_tag.text.replace('€', ''))
                                                else:
                                                    return "not found"
    except:
        return "parser error"

def get_today_ristretto_prices():
    RISTRETTO_PRICE_FILE="ristretto.pkl"
    update = False
    if os.path.exists(RISTRETTO_PRICE_FILE):
        filedate = datetime.fromtimestamp(os.path.getctime(RISTRETTO_PRICE_FILE)).date()
        today = datetime.now().date()
        if filedate == today:
            with open(RISTRETTO_PRICE_FILE, 'rb') as handle:
                return pickle.load(handle)
        else:
            os.remove(RISTRETTO_PRICE_FILE)
            update = True
    else:
        update = True
    
    if update:
        ww_ristreto_prices = []
        today = datetime.today().strftime('%Y-%m-%d')
        for r in nespresso.index:
            local = {}
            local["date"] = today
            local["country"] = nespresso['flag_code'][r]
            local["local_price"] = get_nespresso_price(nespresso['ristretto_url'][r])
            ww_ristreto_prices.append(local)
        with open(RISTRETTO_PRICE_FILE, 'wb') as handle:
            pickle.dump(ww_ristreto_prices, handle, protocol=pickle.HIGHEST_PROTOCOL)
        return ww_ristreto_prices

ristretto=get_today_ristretto_prices()
ALARM

In [3]:
def get_today_fx_rate():
    FX_CACHE_FILE="fx_cache.pkl"
    update_cache = False
    if os.path.exists(FX_CACHE_FILE):
        filedate = datetime.fromtimestamp(os.path.getctime(FX_CACHE_FILE)).date()
        today = datetime.now().date()
        if filedate == today:
            with open(FX_CACHE_FILE, 'rb') as handle:
                return pickle.load(handle)
        else:
            os.remove(FX_CACHE_FILE)
            update_cache = True
    else:
        update_cache = True
    
    if update_cache:
        FX_CACHE = {}
        for currency in set(nespresso.fx.values):
            rate = -1
            req_resp = requests.get(url=GETGEOAPI_URL.format(currency))
            data = req_resp.json()
            if "rates" in data:
                rate = float(data["rates"]["USD"]["rate"])                
            FX_CACHE[currency] = rate
        with open(FX_CACHE_FILE, 'wb') as handle:
            pickle.dump(FX_CACHE, handle, protocol=pickle.HIGHEST_PROTOCOL)
        return FX_CACHE

FX_CACHE = get_today_fx_rate()
ALARM

In [4]:
def calculate_price(FX_CACHE, ristretto, nespresso):
    prices = []
    for row in ristretto:
        entry ={}
        entry["country_code"] = nespresso[nespresso.flag_code==row["country"]].alpha_3.values[0]
        entry["country"] = nespresso[nespresso.flag_code==row["country"]].country.values[0]    
        entry["local_currency"] = nespresso[nespresso.flag_code==row["country"]].fx.values[0]
        price = row["local_price"]*FX_CACHE[entry["local_currency"]]
        if price > 5: # greater tha $5? should be price for 10 units, so ...
            entry["local_price"]= round(row["local_price"]/10,3)
            entry["price"] = round(price/10,2)
        else:
            entry["local_price"]= row["local_price"]
            entry["price"] = round(price,2)
        if (entry['price'] < 4) & (entry['price'] > 0.1):
            prices.append(entry)
    return pd.DataFrame(prices)
prices = calculate_price(FX_CACHE, ristretto, nespresso)
ALARM

In [5]:
prices.head()

,country_code,country,local_currency,local_price,price
0,MUS,Mauritius,MUR,25.0,0.57
1,MAR,Morocco,MAD,5.9,0.63
2,ZAF,South Africa,ZAR,8.5,0.56
3,EGY,Egypt,EGP,14.0,0.89
4,CIV,Ivory Coast,XOF,320.0,0.54


In [6]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
prices = world.merge(prices, how="left", left_on=['iso_a3'], right_on=['country_code'])
prices = prices.dropna(subset=['price'])

In [16]:
nespresso_map = folium.Map()
# Add the data
folium.Choropleth(
    geo_data=prices,
    name='choropleth',
    data=prices,
    columns=['name', 'price'],
    key_on='feature.properties.name',
    fill_color='Accent',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Ristretto Index'
).add_to(nespresso_map)
#Visualize
nespresso_map